In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import LSTM

In [15]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

In [5]:
os.chdir('/content/drive/MyDrive/GAN_basic')

In [351]:
celltrion_train = pd.read_csv('celltrion_train.csv',parse_dates=['date'])

In [352]:
celltrion_test = pd.read_csv('celltrion_test.csv',parse_dates=['date'])

In [353]:
def make_dataset(data, label, window_size=20):
    feature_list = []
    label_list = []
    for i in range(len(data) - window_size):
        feature_list.append(np.array(data.iloc[i:i+window_size]))
        label_list.append(np.array(label.iloc[i+window_size]))
    return np.array(feature_list), np.array(label_list)

In [354]:
celltrion_train.drop(0,axis=0,inplace = True)

In [355]:
celltrion_train = celltrion_train.reset_index(drop=True)

In [356]:
remove_idx = []

for idx,i in enumerate(celltrion_train['celltrion_change']):
    if i == 0:
        celltrion_train['up_count'][idx-1] += celltrion_train['up_count'][idx]
        celltrion_train['down_count'][idx-1] += celltrion_train['down_count'][idx]
        celltrion_train['news_up_count'][idx-1] += celltrion_train['news_up_count'][idx]
        celltrion_train['news_down_count'][idx-1] += celltrion_train['news_down_count'][idx]
        remove_idx.append(idx)
celltrion_train.drop(remove_idx,axis=0,inplace = True)

In [357]:
remove_idx = []

for idx,i in enumerate(celltrion_test['change']):
    if i == 0:
        celltrion_test['up_count'][idx-1] += celltrion_test['up_count'][idx]
        celltrion_test['down_count'][idx-1] += celltrion_test['down_count'][idx]
        celltrion_test['news_up_count'][idx-1] += celltrion_test['news_up_count'][idx]
        celltrion_test['news_down_count'][idx-1] += celltrion_test['news_down_count'][idx]
        remove_idx.append(idx)
celltrion_test.drop(remove_idx,axis=0,inplace = True)

In [358]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler
celltrion_train['celltrion_price'] = MinMaxScaler().fit_transform(celltrion_train['celltrion_price'].values.reshape(-1,1))
celltrion_test['price'] = MinMaxScaler().fit_transform(celltrion_test['price'].values.reshape(-1,1))

In [359]:
celltrion_train.columns

Index(['date', 'up_count', 'down_count', 'celltrion_price', 'celltrion_change',
       'news_up_count', 'news_down_count'],
      dtype='object')

In [360]:
celltrion_train.colunmns = ['date', 'up_count', 'down_count','price', 'change' , 'news_up_count', 'news_down_count']

In [305]:
# celltrion_train.columns = ['date', 'price', 'change', 'up_count', 'down_count', 'news_up_count', 'news_down_count']

In [362]:
celltrion_train['down_count'] = -celltrion_train['down_count']
celltrion_test['down_count'] = -celltrion_test['down_count']

celltrion_train['news_down_count'] = -celltrion_train['news_down_count']
celltrion_test['news_down_count'] = -celltrion_test['news_down_count']

In [370]:
celltrion_train['up'] = celltrion_train['up_count'] + celltrion_train['news_up_count']
celltrion_train['down'] = celltrion_train['down_count'] + celltrion_train['news_down_count']

celltrion_test['up'] = celltrion_test['up_count'] + celltrion_test['news_up_count']
celltrion_test['down'] = celltrion_test['down_count'] + celltrion_test['news_down_count']

celltrion_train.drop(['up_count','news_up_count','down_count','news_down_count'],axis=1,inplace = True)
celltrion_test.drop(['up_count','news_up_count','down_count','news_down_count'],axis=1,inplace = True)

In [308]:
celltrion_train.dropna(inplace = True)

In [371]:
feature_cols = ['price','up','down']
label_cols = ['change']
train_features_col = ['celltrion_price','up','down']
train_label_cols = ['celltrion_change']
train_feature = celltrion_train[train_features_col]
train_label = celltrion_train[train_label_cols]

# train dataset
train_feature, train_label = make_dataset(train_feature, train_label, 1)

# test dataset
test_feature = celltrion_test[feature_cols]
test_label = celltrion_test[label_cols]
test_feature,test_label = make_dataset(test_feature, test_label,1)

In [372]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import LSTM

model = Sequential()
model.add(LSTM(16, 
               input_shape=(train_feature.shape[1], train_feature.shape[2]), 
               activation='relu', 
               return_sequences=False)
          )
model.add(Dense(1))

In [373]:
model.compile(loss='mean_squared_error', optimizer='adam')
early_stop = EarlyStopping(monitor='train_loss', patience=5)
# filename = os.path.join(model_path, 'tmp_checkpoint.h5')
# checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

history = model.fit(train_feature, train_label, 
                    epochs=100, 
                    batch_size=16,
                    validation_data=None, 
                    callbacks=[early_stop])

Epoch 1/100
13/13 [==============================] - 1s 3ms/step - loss: 8.0872
Epoch 2/100
13/13 [==============================] - 0s 3ms/step - loss: 8.0450
Epoch 3/100
13/13 [==============================] - 0s 3ms/step - loss: 8.0062
Epoch 4/100
13/13 [==============================] - 0s 3ms/step - loss: 7.9844
Epoch 5/100
13/13 [==============================] - 0s 3ms/step - loss: 7.9471
Epoch 6/100
13/13 [==============================] - 0s 3ms/step - loss: 7.9171
Epoch 7/100
13/13 [==============================] - 0s 3ms/step - loss: 7.8918
Epoch 8/100
13/13 [==============================] - 0s 3ms/step - loss: 7.8695
Epoch 9/100
13/13 [==============================] - 0s 3ms/step - loss: 7.8421
Epoch 10/100
13/13 [==============================] - 0s 3ms/step - loss: 7.8277
Epoch 11/100
13/13 [==============================] - 0s 4ms/step - loss: 7.7973
Epoch 12/100
13/13 [==============================] - 0s 3ms/step - loss: 7.7803
Epoch 13/100
13/13 [=================

In [374]:
pred = model.predict(test_feature)
pred

array([[-0.1731254 ],
       [-0.24332955],
       [-0.10597648],
       [ 0.00087364],
       [-0.23076165],
       [-0.1604938 ],
       [-0.09450737],
       [ 0.04423378],
       [-0.13969387],
       [-0.13453385]], dtype=float32)

In [375]:
from sklearn.metrics import accuracy_score
pred_c = np.where(pred>0,1,0)
test_label_c = np.where(test_label>0,1,0)

In [376]:
accuracy_score(test_label_c,pred_c)


0.3